In [ ]:
import os, json
import pandas as pd
import math

In [ ]:
# this finds our json files
path_to_json = 'results/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [ ]:
# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['id', 'nodes_in_graph', 'edges_in_graph','total_time', 'nodes_to_block', 'simulation_name'])
json_blocked_data = pd.DataFrame(columns=['simulation_index', 'node'])
idx = 0
offset = 0

# we need both the json and an index number so use enumerate()
for index, file_name in enumerate(json_files):
    tokens = file_name.replace('.json', '').split('_')
    nodes_to_block = int(tokens[1])
    name = tokens[2]
    graph_stats = tokens[3].split(']')
    num_nodes = int(graph_stats[0].replace('[',''))
    num_edges = int(graph_stats[1].replace('[',''))

    with open(os.path.join(path_to_json, file_name)) as json_file:
        json_text = json.load(json_file)
        jsons_data.loc[index] = [index + offset, num_nodes, num_edges, json_text['Total time'], nodes_to_block, name]

        for blocked_node in json_text['Blocked nodes']:
            json_blocked_data.loc[idx] = [index + offset, blocked_node]
            idx += 1

In [ ]:
jsons_data.to_csv('results/simulationData.csv', index=False)
json_blocked_data.to_csv('results/simulationNodes.csv', index=False)

In [ ]:
path_to_json = 'outputs/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [ ]:
sim_data = pd.DataFrame(columns=['simulation_index', 'mean_activated', 'var_activated', 'mean_saved', 'var_saved', 'saved_nodes', 'activated_nodes'])

In [ ]:
index = 0
for _, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        for sim_id in json_text['solvers'].keys():
            mean_activated = json_text['solvers'][sim_id]['activated nodes']['mean']
            var_activated = json_text['solvers'][sim_id]['activated nodes']['var']
            mean_saved = json_text['solvers'][sim_id]['saved nodes']['mean']
            var_saved = json_text['solvers'][sim_id]['saved nodes']['var']

            sim_data.loc[index] = [sim_id, mean_activated, var_activated, mean_saved, mean_activated, int(math.ceil(mean_saved)), int(math.ceil(mean_activated))]
            index += 1

In [ ]:
sim_data.to_csv('results/simulationStats.csv', index=False)